In [66]:
import pandas as pd 

import numpy as np
import multiprocessing as mp
import functools
from string import ascii_lowercase
import timeit
import itertools
import cProfile

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import itertools

import os
import sys

# Preparation!! Merge context of all files in different directory
### First of all, I want to collect all the path of file that end with **"-plain.txt"** in the EN directory

In [67]:
path = '/mnt/d/Git/Repositories/IRPython/src/handOn4/EN'
os.chdir(path)
directories_in_EN = os.listdir()
target_file_lists = []

for files in directories_in_EN:
    folder = f'/mnt/d/Git/Repositories/IRPython/src/handOn4/EN/{files}'
    os.chdir(folder)
    filesList = os.listdir()
    for names in filesList:
        if names.endswith('-plain.txt'):
            target_file = f"{folder}/{names}"
    target_file_lists.append(target_file)



### After we get all the path of file i decide to read all of it and write in the output file that i choosen.

In [68]:
all_contents = '/mnt/d/Git/Repositories/IRPython/src/handOn4/opt.txt'
with open(all_contents, "w") as outfile:
    for filename in target_file_lists:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

In [69]:
f = open(all_contents, 'r')
context = f.read()
print(context[1:500])

Strategies on the future of the chlorine industry 

C. Hendriks and D. Papameletiou 

Issue: 

Production and use of chlorine are of key importance for the chemical industry, with a large number of chlorinated compounds on the market, and nearly 60% of the global sales of chemical products being directly or indirectly linked with chlorine.
Besides its importance for the economy, chlorine has for many years been the subject of controversial debates because of adverse environmental and health imp


### DONE Preparation

# Task No.1 Let's clean the data from reading

### First, delete all Non-English charecter

In [70]:
import re
context_no_non_eng = re.sub("[^A-Za-z]", " ", context)
print(context_no_non_eng[1:500])

Strategies on the future of the chlorine industry   C  Hendriks and D  Papameletiou   Issue    Production and use of chlorine are of key importance for the chemical industry  with a large number of chlorinated compounds on the market  and nearly     of the global sales of chemical products being directly or indirectly linked with chlorine  Besides its importance for the economy  chlorine has for many years been the subject of controversial debates because of adverse environmental and health imp


### Secound, Covert too many whitespce into one

In [71]:
context_nonEng_single_spcace = re.sub('\s+', ' ', context_no_non_eng)
print(context_nonEng_single_spcace[1:500])

Strategies on the future of the chlorine industry C Hendriks and D Papameletiou Issue Production and use of chlorine are of key importance for the chemical industry with a large number of chlorinated compounds on the market and nearly of the global sales of chemical products being directly or indirectly linked with chlorine Besides its importance for the economy chlorine has for many years been the subject of controversial debates because of adverse environmental and health impacts of many chlo


### Lastly, Cast every letter to lowercase

In [72]:
clean_context = context_nonEng_single_spcace.lower()
print(clean_context[1:500])

strategies on the future of the chlorine industry c hendriks and d papameletiou issue production and use of chlorine are of key importance for the chemical industry with a large number of chlorinated compounds on the market and nearly of the global sales of chemical products being directly or indirectly linked with chlorine besides its importance for the economy chlorine has for many years been the subject of controversial debates because of adverse environmental and health impacts of many chlo


### Done 1st task

# Task No.2 Create the table of term and freq as the page #22

In [73]:
IULA = pd.DataFrame(
    [
        ['defeat', clean_context.count('defeat')],
        ['decet', clean_context.count('decet')],
        ['defect', clean_context.count('defect')],
        ['deft', clean_context.count('deft')],
        ['defer', clean_context.count('defer')],
        ['Deeft', clean_context.count('Deeft')]
    ],
    columns=['word', 'frequency']
)
IULA_pop = 2.1e6
IULA['P(w)'] = IULA['frequency']/IULA_pop
IULA['rank'] = IULA['frequency'].rank(ascending=False).astype(int)

IULA

,word,frequency,P(w),rank
0,defeat,12,0.000006,2
1,decet,0,0.000000,5
2,defect,198,0.000094,1
3,deft,0,0.000000,5
4,defer,11,0.000005,3
5,Deeft,0,0.000000,5


For my practicing and the example on hand on, it is **differnt**.

From my hypotheses, refer from the $ P(w) = \frac{C(w)}{T} $

In my frequency $ (C(w)) $ is **not the same** as the previous. I guess because I do on the real data but in the slide should make it simple.

## Let do 10^9P(x|w)P(w) by using Norvig's prob

### Prepare norvig's prob

In [74]:
norvig = pd.read_csv('http://norvig.com/ngrams/count_1edit.txt',
                     sep='\t', encoding="ISO-8859-1", header=None)
norvig.columns = ['term', 'edit']
norvig = norvig.set_index('term')
norvig.head()

,edit
term,
e|i,917
a|e,856
i|e,771
e|a,749
a|i,559


In [75]:
norvig_orig = pd.read_csv('http://norvig.com/ngrams/count_big.txt',
                          sep='\t', encoding="ISO-8859-1", header=None)
norvig_orig = norvig_orig.dropna()
norvig_orig.columns = ['term', 'freq']
norvig_orig.head()


,term,freq
0,a,21160
1,aah,1
2,aaron,5
3,ab,2
4,aback,3


In [76]:
def get_count(c, norvig_orig):
    return norvig_orig.apply(lambda x: x.term.count(c) * x.freq, axis=1).sum()

In [77]:
character_set = list(map(''.join, itertools.product(ascii_lowercase, repeat=1))) + \
    list(map(''.join, itertools.product(ascii_lowercase, repeat=2)))

In [78]:
with mp.Pool(processes=8) as pool:
    freq_list = pool.map(functools.partial(
        get_count, norvig_orig=norvig_orig), character_set)

freq_df = pd.DataFrame([character_set, freq_list], index=['char', 'freq']).T
freq_df = freq_df.set_index('char')

### Done of set up
* freq_df as frequency per character

### Set the new coulumn as $ P(x|w) $

In [79]:
IULA['P(x|w)'] = [
    (norvig.loc['e|ea'].values / freq_df.loc['ea'].values)[0],
    (norvig.loc['f|c'].values / freq_df.loc['c'].values)[0],
    (norvig.loc['e|ec'].values / freq_df.loc['ec'].values)[0],
    (norvig.loc['e| '].values / freq_df.loc['e'].values)[0],
    (norvig.loc['t|r'].values / freq_df.loc['r'].values)[0],
    (norvig.loc['fe|ef'].values / freq_df.loc['ef'].values)[0]
]
IULA

,word,frequency,P(w),rank,P(x|w)
0,defeat,12,0.000006,2,0.012834
1,decet,0,0.000000,5,0.000028
2,defect,198,0.000094,1,0.003167
3,deft,0,0.000000,5,0.000003
4,defer,11,0.000005,3,0.000036
5,Deeft,0,0.000000,5,0.003311


### Add the new column as $ 10^9 P(x|w)P(w) $

In [80]:
IULA['10e9 P(x|w)P(w)'] = 1e9 * IULA['P(w)'] * IULA['P(x|w)']
IULA

,word,frequency,P(w),rank,P(x|w),10e9 P(x|w)P(w)
0,defeat,12,0.000006,2,0.012834,73.337097
1,decet,0,0.000000,5,0.000028,0.000000
2,defect,198,0.000094,1,0.003167,298.593664
3,deft,0,0.000000,5,0.000003,0.000000
4,defer,11,0.000005,3,0.000036,0.186141
5,Deeft,0,0.000000,5,0.003311,0.000000


### Done Task No.2

# Task No.3 Calculate the Incorporating context

### In the sentense “It is difficult to make **a defet-free** software product"

### To predict that incorrect group of word, which one is should be more candidate between **a defeat free** or **a defect free** 

### So we need group **words** as the slide in page37

* a defeat
* a defect
* defeat free
* defect free

### Let's start on create the table

In [81]:
IULA_calcu = pd.DataFrame(
    [['a defeat', 'P(defeat|a)','C(a defeat)/C(defeat)',clean_context.count('a defeat')/clean_context.count('defeat')],
     ['a defect', 'P(defect|a)','C(a defeat)/C(defeat)',clean_context.count('a defect')/clean_context.count('defect')],
     ['defeat free','P(free|defeat)','C(defeat free)/C(free)',clean_context.count('defeat free')/clean_context.count('free')],
     ['defect free','P(free|defect)','C(defect free)/C(free)',clean_context.count('defect free')/clean_context.count('defect')]
     ],
    columns=['group of word', 'P(Wk|Wk-1)', 'C(Wk-1|Wk)/C(Wk-1)', 'Evaluate']
)
IULA_calcu


,group of word,P(Wk|Wk-1),C(Wk-1|Wk)/C(Wk-1),Evaluate
0,a defeat,P(defeat|a),C(a defeat)/C(defeat),0.000000
1,a defect,P(defect|a),C(a defeat)/C(defeat),0.106061
2,defeat free,P(free|defeat),C(defeat free)/C(free),0.000000
3,defect free,P(free|defect),C(defect free)/C(free),0.000000


### In the **group of word** the **first word** refer as $ W _{k-1} $ and the **second** as $ W _k $
### Refer to the first row, we have group of word as **a defeat** by $ W _k $ = **a** and $ W _{k-1} $ = **defeat**
### but in the thrid row, we have group of word as **defeat free** so the **defeat** is $ W _{k-1} $ and the **free** is $ W _k $

In [82]:
P_1 = IULA_calcu['Evaluate'][0]*IULA_calcu['Evaluate'][2]
print("P(“a defeat free”) = " + str(P_1))

P(“a defeat free”) = 0.0


In [83]:
P_2 = IULA_calcu['Evaluate'][1]*IULA_calcu['Evaluate'][3]
print("P(“a defect free”) = " + str(P_2))
IULA_calcu['Evaluate'][1]

P(“a defect free”) = 0.0


0.10606060606060606

## Conclusion, with IULA data we cannot found the which is the candidate of correct word. Because of the score of **P(“a defeat free”)** and **"P(“a defect free”)** is 0. But in the class we can say the "P(“a defeat free”) is the most candidate because it has Candidate value more that another.

## Done the last part